In [1]:
from obspy import read
from obspy.core.util import get_example_file
import numpy as np
import pandas as pd
import segyio
import matplotlib.pyplot as plt

In [56]:
filename = "../data/raw/MAX_SGY/For_Masw_Resample.sgy"
with segyio.open(filename, "r", endian='big', ignore_geometry=True) as segyfile:
    print(len(segyfile.trace))
    # for i in range(0,33578-33,33):
    #     if ((segyfile.header[i][segyio.TraceField.GroupX]/100 + segyfile.header[i+32][segyio.TraceField.GroupX]/100) / 2 > 468980.19 - 1) and ((segyfile.header[i][segyio.TraceField.GroupX]/100 + segyfile.header[i+32][segyio.TraceField.GroupX]/100) / 2 < 468980.19 + 1):
    #         print(segyfile.header[i+32][segyio.TraceField.FieldRecord])
    #         print(((segyfile.header[i][segyio.TraceField.GroupX]/100 + segyfile.header[i+32][segyio.TraceField.GroupX]/100) / 2 ))
    #         print(((segyfile.header[i][segyio.TraceField.GroupY]/100 + segyfile.header[i+32][segyio.TraceField.GroupY]/100) / 2 ))
#FieldRecord - FFID 1 сортировка
#TraceNumber - CHAN 2 сортировка
#SourceX, SourceY - координаты источника * SourceGroupScalar
#GroupX, SourceY - координаты приемника * ElevationScalar
#10000

1108074


In [30]:
filename = "../data/raw/MAX_SGY/For_Masw_Resample.sgy"
with segyio.open(filename, "r", endian='big', ignore_geometry=True) as segyfile:
    print(segyfile.header[0])
    print(segyfile.header[33])
#FieldRecord - FFID 1 сортировка
#TraceNumber - CHAN 2 сортировка
#SourceX, SourceY - координаты источника * SourceGroupScalar
#GroupX, SourceY - координаты приемника * ElevationScalar

{TRACE_SEQUENCE_LINE: 30655, TRACE_SEQUENCE_FILE: 0, FieldRecord: 10102414, TraceNumber: 865, EnergySourcePoint: 0, CDP: 51050964, CDP_TRACE: 0, TraceIdentificationCode: 1, NSummedTraces: 0, NStackedTraces: 1, DataUse: 0, offset: 102, ReceiverGroupElevation: 700, SourceSurfaceElevation: 0, SourceDepth: 600, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 1979, GroupWaterDepth: 0, ElevationScalar: -100, SourceGroupScalar: -100, SourceX: 46935710, SourceY: 769618660, GroupX: 46928220, GroupY: 769611690, CoordinateUnits: 1, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 4, GroupStaticCorrection: 4, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 438, TRACE_SAMPLE_INTERVAL: 16000, GainType: 3, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, Swee

In [83]:
filename = "../data/raw/2023_MSU_MASW/02.Data/03.Result/1_UR203D1004I1211_fund.sgy"
with segyio.open(filename, "r", endian='big', ignore_geometry=True) as segyfile:
    # Читаем заголовки трасс
    print(segyfile.header[0])
    print('----------')
    print(segyfile.trace[0])
#FieldRecord - FFID 1 сортировка
#TraceNumber - CHAN 2 сортировка
#SourceX, SourceY - координаты источника * SourceGroupScalar
#GroupX, SourceY - координаты приемника * ElevationScalar

{TRACE_SEQUENCE_LINE: 1, TRACE_SEQUENCE_FILE: 0, FieldRecord: 1, TraceNumber: 1, EnergySourcePoint: 0, CDP: 1, CDP_TRACE: 1, TraceIdentificationCode: -1, NSummedTraces: 0, NStackedTraces: 0, DataUse: 0, offset: 0, ReceiverGroupElevation: 0, SourceSurfaceElevation: 0, SourceDepth: 0, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 0, GroupWaterDepth: 0, ElevationScalar: -100, SourceGroupScalar: -100, SourceX: 46916559, SourceY: 0, GroupX: 46916559, GroupY: 0, CoordinateUnits: 1, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 0, GroupStaticCorrection: 0, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 10, TRACE_SAMPLE_INTERVAL: 10000, GainType: 0, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, SweepType: 0, SweepTraceTaperLengthStart: 0, Swee

In [58]:
def coord_func(x):
    """
    Function to calculate the coordinate of a central point
    :param x: dataframe
    :return: coordinate of the central point (dataframe)
    """
    return pd.DataFrame(
        [[(np.min(x['REC_X']) + np.max(x['REC_X'])) / 2, (np.min(x['REC_Y']) + np.max(x['REC_Y'])) / 2]],
        columns=["Receiver Midpoint_X", "Receiver Midpoint_Y"])
def segy2df(filename):
    """
    Function to read a segy and return df with coordinates of central points
    :param filename: path to segy file
    :return: dataframe with coordinates of central points
    """
    base = {'FFID': [], 'CHAN': [], "REC_X": [], "REC_Y": []}
    try:
        with segyio.open(filename, "r", ignore_geometry=True) as segyfile:
            headers = segyfile.header
            for header in headers:
                base['FFID'].append(header[segyio.TraceField.FieldRecord])
                base['CHAN'].append(header[segyio.TraceField.TraceNumber])
                base['REC_X'].append(header[segyio.TraceField.GroupX])
                base['REC_Y'].append(header[segyio.TraceField.GroupY])
    except:
        with segyio.open(filename, "r", endian='little') as segyfile:
            headers = segyfile.header
            for header in headers:
                base['FFID'].append(header[segyio.TraceField.FieldRecord])
                base['CHAN'].append(header[segyio.TraceField.TraceNumber])
                base['REC_X'].append(header[segyio.TraceField.GroupX])
                base['REC_Y'].append(header[segyio.TraceField.GroupY])
    df = pd.DataFrame(base)
    return df.groupby('FFID').apply(coord_func)

In [121]:
df = segy2df('../data/raw/SGY_10/Profile.sgy')

C:\Users\vitya\AppData\Local\Temp\ipykernel_9912\849246695.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('FFID').apply(coord_func)


In [122]:
df.head(15)

,,Receiver Midpoint_X,Receiver Midpoint_Y
FFID,,,
1177,0,4889226.5,76563934.0
1191,0,4890626.0,76566155.0
1205,0,4892036.5,76568368.5
1219,0,4893477.5,76570569.5
1233,0,4894884.5,76572785.5
1247,0,4896271.5,76574998.5
1261,0,4897741.0,76577203.5
1275,0,4899200.5,76579387.0
1289,0,4900645.0,76581587.5


In [123]:

df['FFID'] = df.index.to_series().apply(lambda x: x[0]).values

In [124]:
df

,,Receiver Midpoint_X,Receiver Midpoint_Y,FFID
FFID,,,,
1177,0,4889226.5,76563934.0,1177
1191,0,4890626.0,76566155.0,1191
1205,0,4892036.5,76568368.5,1205
1219,0,4893477.5,76570569.5,1219
1233,0,4894884.5,76572785.5,1233
...,...,...,...,...
2171,0,4989537.0,76720934.5,2171
2185,0,4991033.0,76723102.5,2185
2199,0,4992512.0,76725274.0,2199


In [125]:
def merge_df(coord_df, filepath):
    """
    Merge two dataframes (coord and depth) into a single
    :param coord_df: dataframe containing coordinate of central point
    :param filepath: filepath to .txt file with depths
    :return: merged dataframe
    """
    profile_txt = pd.read_csv(filepath, delimiter="\t")
    profile_txt.rename(columns={"Receiver Midpoint": "Receiver Midpoint_X"}, inplace=True)
    coord_df["Receiver Midpoint_X"] = coord_df["Receiver Midpoint_X"] / 10
    coord_df["Receiver Midpoint_Y"] = coord_df["Receiver Midpoint_Y"] / 10
    df_last = pd.merge(coord_df, profile_txt, left_on='Receiver Midpoint_X', right_on='Receiver Midpoint_X')
    df_last['Depth'] = -df_last['Depth']
    df_last.drop(['REC_STAT1', 'REC_STAT2', 'Vs30'], axis=1, inplace=True)
    return df_last

In [126]:
last_df = merge_df(df, '../data/raw/TXT_10/Profile.txt')

In [127]:
last_df

,Receiver Midpoint_X,Receiver Midpoint_Y,FFID,Depth,Velocity
0,488922.65,7656393.40,1177,-4.889,131.861
1,488922.65,7656393.40,1177,-14.667,143.536
2,488922.65,7656393.40,1177,-24.444,230.461
3,488922.65,7656393.40,1177,-34.222,279.696
4,488922.65,7656393.40,1177,-44.000,297.708
...,...,...,...,...,...
755,499533.95,7672964.65,2227,-53.778,389.716
756,499533.95,7672964.65,2227,-63.556,343.989
757,499533.95,7672964.65,2227,-73.333,327.319
758,499533.95,7672964.65,2227,-83.111,314.909


In [129]:
grouped_df = last_df.groupby(['Receiver Midpoint_X', 'Receiver Midpoint_Y'])
grouped_df.value_counts()

Receiver Midpoint_X  Receiver Midpoint_Y  FFID  Depth    Velocity
488922.65            7656393.40           1177  -92.400  305.335     1
                                                -83.111  305.316     1
                                                -73.333  305.230     1
                                                -63.556  304.863     1
                                                -53.778  303.326     1
                                                                    ..
499533.95            7672964.65           2227  -44.000  338.023     1
                                                -34.222  279.475     1
                                                -24.444  185.256     1
                                                -14.667  168.014     1
                                                -4.889   158.284     1
Name: count, Length: 760, dtype: int64

In [135]:
def txt2sgy(input_df, output_filepath: str) -> None:
    grouped_df = input_df.groupby(['Receiver Midpoint_X', 'Receiver Midpoint_Y', 'FFID'])
    max_samples = max(group.shape[0] for name, group in grouped_df)
    spec = segyio.spec()
    spec.ilines = np.arange(1, len(grouped_df)+1)
    spec.xlines = np.arange(1)
    spec.samples = np.arange(max_samples)
    spec.format = 1

    with segyio.create(output_filepath, spec) as f:
        f.text[0] = segyio.tools.wrap("Created with segyio")
        f.bin.update({
            segyio.BinField.Interval: 100,
            segyio.BinField.Samples: max_samples,
            segyio.BinField.Format: 1,
        })

        trace_idx = 0

        for (x, y, ffid), group in grouped_df:
            group = group.sort_values(by='Depth', ascending=False)

            trace_data = np.zeros(max_samples, dtype=np.float32)
            trace_data[:len(group['Velocity'])] = group['Velocity'].values

            f.trace[trace_idx] = trace_data

            f.header[trace_idx].update({
                segyio.TraceField.SourceGroupScalar: -100,
                segyio.TraceField.ElevationScalar: -100,
                segyio.TraceField.CDP_X: int(x*100),
                segyio.TraceField.CDP_Y: int(y*100),
                segyio.TraceField.SourceX: int(x * 100),
                segyio.TraceField.SourceY: int(y * 100),
                segyio.TraceField.TRACE_SAMPLE_COUNT: max_samples,
                segyio.TraceField.TRACE_SAMPLE_INTERVAL: 10000,
                segyio.TraceField.FieldRecord: ffid,
                segyio.TraceField.TraceNumber: trace_idx+1

            })

            trace_idx += 1

In [136]:
txt2sgy(last_df, '../data/interim/OUTPUT/Profile.sgy')

In [139]:
filename = "../data/raw/MAX_SGY/For_Masw_Resample.sgy"
with segyio.open('../data/interim/OUTPUT/Profile.sgy', "r", endian='big', ignore_geometry=True) as segyfile:
    print(segyfile.header[3])
#FieldRecord - FFID 1 сортировка
#TraceNumber - CHAN 2 сортировка
#SourceX, SourceY - координаты источника * SourceGroupScalar
#GroupX, SourceY - координаты приемника * ElevationScalar

{TRACE_SEQUENCE_LINE: 0, TRACE_SEQUENCE_FILE: 0, FieldRecord: 1219, TraceNumber: 4, EnergySourcePoint: 0, CDP: 0, CDP_TRACE: 0, TraceIdentificationCode: 0, NSummedTraces: 0, NStackedTraces: 0, DataUse: 0, offset: 0, ReceiverGroupElevation: 0, SourceSurfaceElevation: 0, SourceDepth: 0, ReceiverDatumElevation: 0, SourceDatumElevation: 0, SourceWaterDepth: 0, GroupWaterDepth: 0, ElevationScalar: -100, SourceGroupScalar: -100, SourceX: 48934775, SourceY: 765705695, GroupX: 0, GroupY: 0, CoordinateUnits: 0, WeatheringVelocity: 0, SubWeatheringVelocity: 0, SourceUpholeTime: 0, GroupUpholeTime: 0, SourceStaticCorrection: 0, GroupStaticCorrection: 0, TotalStaticApplied: 0, LagTimeA: 0, LagTimeB: 0, DelayRecordingTime: 0, MuteTimeStart: 0, MuteTimeEND: 0, TRACE_SAMPLE_COUNT: 10, TRACE_SAMPLE_INTERVAL: 10000, GainType: 0, InstrumentGainConstant: 0, InstrumentInitialGain: 0, Correlated: 0, SweepFrequencyStart: 0, SweepFrequencyEnd: 0, SweepLength: 0, SweepType: 0, SweepTraceTaperLengthStart: 0, S